<a href="https://colab.research.google.com/github/ja8an/big-data/blob/main/TDE_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!wget -P -Q https://github.com/ja8an/big-data/blob/main/in/transactions_amostra.csv?raw=true -O transactions.csv

--2022-11-16 16:56:57--  https://github.com/ja8an/big-data/blob/main/in/transactions_amostra.csv?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/ja8an/big-data/raw/main/in/transactions_amostra.csv [following]
--2022-11-16 16:56:57--  https://github.com/ja8an/big-data/raw/main/in/transactions_amostra.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/ja8an/big-data/main/in/transactions_amostra.csv [following]
--2022-11-16 16:56:57--  https://media.githubusercontent.com/media/ja8an/big-data/main/in/transactions_amostra.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.1

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [19]:
schema = 'country_or_area string,year string,comm_code long,commodity string,flow string,trade_usd double,weight_kg double,quantity_name string,quantity double,category string'
transactions = spark.read.option("Header", True).csv('/content/transactions.csv', schema=schema, sep=';')

In [32]:
transactions.head()

Row(country_or_area='Belgium', year=2016, comm_code=920510, commodity='Brass-wind instruments', flow='Export', trade_usd=571297.0, weight_kg=3966.0, quantity_name='Number of items', quantity=4135.0, category='92_musical_instruments_parts_and_accessories')

(Easy) The number of transactions involving Brazil;

In [23]:
transactions[transactions.country_or_area == 'Brazil'].count()

54762

(Easy) The number of transactions per year;

In [28]:
transactions.groupBy('year').count().toPandas()

,year,count
0,2007,105891
1,2014,103231
2,1988,8568
3,2012,102989
4,1991,22652
5,2016,82277
6,1994,54143
7,1999,86191
8,1997,77876
9,2009,105686


(Easy) The number of transactions per flow type and year;

In [31]:
transactions.groupBy('year','flow').count().orderBy("year", asc=True).toPandas()

,year,flow,count
0,1988,Import,4268
1,1988,Re-Export,758
2,1988,Export,3542
3,1989,Export,7092
4,1989,Re-Export,979
...,...,...,...
99,2015,Import,57960
100,2016,Re-Import,1966
101,2016,Import,45236
102,2016,Export,30538


(Easy) The average of commodity values per year;

In [33]:
from pyspark.sql.functions import avg
transactions.groupBy("year").agg(avg("trade_usd")).toPandas()

,year,avg(trade_usd)
0,2007,2.589202e+07
1,2014,4.077148e+07
2,1988,1.675092e+07
3,2012,3.699382e+07
4,1991,1.206917e+07
5,2016,2.867160e+07
6,1994,1.153660e+07
7,1999,9.328194e+06
8,1997,1.055229e+07
9,2009,2.568866e+07


(Easy) The average price of commodities per unit type, year, and category in the export flow in Brazil;

In [35]:
transactions.groupBy("quantity_name","year","category").agg(avg("trade_usd")).orderBy("year", asc=True).toPandas()

,quantity_name,year,category,avg(trade_usd)
0,Weight in kilograms,1988,16_meat_fish_and_seafood_food_preparations_nes,1.481900e+07
1,Weight in kilograms,1988,76_aluminium_and_articles_thereof,7.119730e+07
2,Weight in kilograms,1988,24_tobacco_and_manufactured_tobacco_substitutes,2.183376e+07
3,Volume in litres,1988,08_edible_fruit_nuts_peel_of_citrus_fruit_melons,7.171650e+05
4,Volume in litres,1988,22_beverages_spirits_and_vinegar,1.677577e+07
...,...,...,...,...
5029,Number of items,2016,84_nuclear_reactors_boilers_machinery_etc,1.384300e+07
5030,Weight in kilograms,2016,93_arms_and_ammunition_parts_and_accessories_t...,8.290895e+06
5031,Weight in kilograms,2016,95_toys_games_sports_requisites,9.887504e+07
5032,Weight in kilograms,2016,23_residues_wastes_of_food_industry_animal_fodder,4.633019e+07


(Medium) The maximum, minimum, and mean transaction price per unit type and year;

In [36]:
from pyspark.sql.functions import max, min, mean

transactions.groupBy("quantity_name","year").agg(max("trade_usd"),min("trade_usd"),mean("trade_usd")).orderBy("year", asc=True).toPandas()

,quantity_name,year,max(trade_usd),min(trade_usd),avg(trade_usd)
0,Volume in litres,1988,1.842330e+10,6.0,8.903355e+07
1,Area in square metres,1988,4.033860e+08,7.0,6.958580e+06
2,Weight in kilograms,1988,5.171220e+09,7.0,1.480394e+07
3,Number of items,1988,9.339513e+08,62.0,1.370974e+07
4,Number of pairs,1988,2.914562e+07,1069.0,2.968404e+06
...,...,...,...,...,...
263,Number of items,2016,3.544448e+10,1.0,3.581437e+07
264,Weight in carats,2016,9.557468e+06,116633.0,4.496361e+06
265,Weight in kilograms,2016,5.686561e+10,1.0,2.796230e+07
266,Electrical energy in thousands of kilowatt-hours,2016,1.065283e+09,1128262.0,1.765358e+08


(Hard) The most commercialized commodity (summing the quantities) in 2016, per flow type.

In [65]:
from pyspark.sql import Window
from pyspark.sql.functions import col, sum

w = Window.partitionBy("flow")
transactions.groupBy("flow", "commodity") \
.agg(sum("quantity").alias("sum_quantity")) \
.withColumn("max_quantity", max("sum_quantity").over(w)) \
.where(col("max_quantity") == col("sum_quantity")) \
.toPandas()

,flow,commodity,sum_quantity,max_quantity
0,Export,Cigarettes containing tobacco,1.028865e+15,1.028865e+15
1,Import,"Petroleum oils, oils from bituminous minerals,...",9.551994e+12,9.551994e+12
2,Re-Export,Cigarettes containing tobacco,8.388470e+10,8.388470e+10
3,Re-Import,"Granules, chippings, powder of granite,sandsto...",2.700850e+09,2.700850e+09
